In [1]:
using Plots
using Revise
if splitdir(pwd())[2] == "julia"
    cd("..")
end
include("julia/test_definitions.jl")
using KelpTest

In [2]:
using Interpolations

In [3]:
using Cubature

In [4]:
import KelpTest.rte1d_exact
import KelpTest.asymptotics1d_exact

In [5]:
∫(x...) = hquadrature(x...)[1]

∫ (generic function with 1 method)

In [6]:
test_fun(1,2,3)

(1, 2, 3)


In [7]:
function rte1d_exact(I₀, a, b, zmin, zmax, z)
    z₀, z₁ = zmin, zmax
    s = a/b + 1
    q = sqrt(a^2+2*a*b)/b
    c₂ = -(s+q)*I₀*exp(b*q*(2*z₁-z₀))/(s-q-(s+q)*exp(2b*q*(z₁-z₀)))
    c₁ = exp(-b*q*z₀) * (I₀ - c₂*exp(-b*q*z₀))
    L⁺ = c₁*exp.(b*q*z) + c₂*exp.(-b*q*z)
    L⁻ = c₁*(s+q)*exp.(b*q*z) + c₂*(s-q)*exp.(-b*q*z)
    return L⁺, L⁻
end
export rte1d_exact

In [8]:
function asymptotics1d_exact(I₀::Float64, a::Float64, b::Float64, 
        zmin::Float64, zmax::Float64, z::Array{Float64,1}, num_scatters::Int)
    # Radiance without scattering
    L₀⁺ = I₀ * exp.(-a*(z.-zmin))
    L₀⁻ = zeros(z)

    # Radiance to use for source calculation
    Lₙ₋₁⁺ = L₀⁺[:]
    Lₙ₋₁⁻ = L₀⁻[:]

    # Initialize final radiance
    L⁺ = L₀⁺[:]
    L⁻ = L₀⁻[:]

    for n = 1:num_scatters
        # Calculate source
        gₙ⁺ = interpolate(Lₙ₋₁⁻ - Lₙ₋₁⁺, BSpline(Linear()), OnCell())
        gₙ⁻ = interpolate(Lₙ₋₁⁺ - Lₙ₋₁⁻, BSpline(Linear()), OnCell())

        # Define nested anonymous functions and apply elementwize to z
        Lₙ⁺ = (z -> ∫(zp -> exp(-a*(z-zp)) * gₙ⁺[zp], zmin, z)).(z)
        Lₙ⁻ = (z -> ∫(zp -> exp(-a*(zp-z)) * gₙ⁻[zp], z, zmax)).(z)

        # Accumulate total radiance
        L⁺ += b^n * Lₙ⁺
        L⁻ += b^n * Lₙ⁻

        # Update
        Lₙ₋₁⁺[:] = Lₙ⁺[:]
        Lₙ₋₁⁻[:] = Lₙ⁻[:]
    end

    return L⁺, L⁻
end
export asymptotics1d_exact


In [9]:
zmin = 0.0
zmax = 1.0
nz = 101
I₀ = 0.92
a = 1.0
b = 0.5
# Remaining in current direction
β₀ = 0.0
# Switching directions
βπ = 1 - β₀
# Attenuation
c = -(a+(1-β₀)*b)
# Scattering gain
d = b*βπ
z = collect(linspace(zmin, zmax, nz))
num_scatters = 4

4

In [10]:
length

length (generic function with 117 methods)

In [11]:
L⁺e, L⁻e = rte1d_exact(I₀, a, b*βπ, zmin, zmax, z);
L⁺a, L⁻a = asymptotics1d_exact(I₀, a, b*βπ, zmin, zmax, z, num_scatters);
zg, L⁺g, L⁻g = asymptotics1d_grid(2π*I₀, a, b, zmin, zmax, nz, num_scatters);
plot(z, L⁺e, label="L+e")
plot!(z, L⁻e, label="L-e")
plot!(z, L⁺a, marker=:square, label="L+a")
plot!(z, L⁻a, marker=:square, label="L-a")
plot!(zg, L⁺g, marker=:circle, label="L+g")
plot!(zg, L⁻g, marker=:circle, label="L-g")

LoadError: [91mMethodError: no method matching asymptotics1d_grid(::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Int64, ::Int64)[0m
Closest candidates are:
  asymptotics1d_grid(::Any, ::Any, ::Any, ::Any, ::Any, ::Any, ::Any, [91m::Any[39m) at /home/oliver/academic/research/kelp/julia/test_definitions.jl:390[39m

In [12]:
plot(z, L⁺g, marker=:circle, label="L+g")
plot!(z, L⁻g, marker=:circle, label="L-g")
plot!(z, L⁺e, marker=:circle, label="L+e")
plot!(z, L⁻e, marker=:circle, label="L-e")

LoadError: [91mUndefVarError: L⁺g not defined[39m